In [1]:
from datetime import datetime

import pandas as pd
from prophet import Prophet

import plotly.graph_objects as go

In [5]:
df_tmp = pd.read_csv('../data/INV_Transacton_Summary_merge_filled_0.csv', index_col=0)
df_tmp

,YYYYMM,IEM_CODE,ITEM_DESCRIPTION,PO_RECEIPT,SPG,UOM,OTHER_ISSUE,OTHER_RECEIPT,SALES_ISSUE,WIP_ISSUE,WIP_RECEIPT
0,201401,09010003350,"IC,LIN,REF192ES,SMD,R,ANALOG DEVICE,MSL1",3528.0,500,ea,1980.0,1960.0,1960.0,160.0,0.0
1,201402,09010003350,"IC,LIN,REF192ES,SMD,R,ANALOG DEVICE,MSL1",1568.0,500,ea,1862.0,1862.0,1862.0,0.0,0.0
2,201403,09010003350,"IC,LIN,REF192ES,SMD,R,ANALOG DEVICE,MSL1",2744.0,500,ea,1568.0,1568.0,1568.0,0.0,0.0
3,201404,09010003350,"IC,LIN,REF192ES,SMD,R,ANALOG DEVICE,MSL1",2450.0,500,ea,2450.0,2450.0,2450.0,96.0,0.0
4,201405,09010003350,"IC,LIN,REF192ES,SMD,R,ANALOG DEVICE,MSL1",392.0,500,ea,2254.0,2254.0,2254.0,20.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
71290,202109,08910001267,"TR,PIM,FP100R07N3E4,650V,100A,B,INFINEON",0.0,550,ea,0.0,6.0,0.0,6.0,0.0
71291,202110,08910001267,"TR,PIM,FP100R07N3E4,650V,100A,B,INFINEON",0.0,550,ea,0.0,0.0,0.0,0.0,0.0
71292,202111,08910001267,"TR,PIM,FP100R07N3E4,650V,100A,B,INFINEON",0.0,550,ea,0.0,0.0,0.0,0.0,0.0
71293,202112,08910001267,"TR,PIM,FP100R07N3E4,650V,100A,B,INFINEON",0.0,550,ea,0.0,0.0,0.0,0.0,0.0
